In [1]:
import sys
sys.path.append('..')

import json
import pandas as pd

## Extract base PhD dataset

In [2]:
dataset_path = '../data/phd/phd.json'
save_path = '../data/phd/phd_base.json'
sample_size = None

with open(dataset_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)
df = pd.DataFrame(dataset)
# Skip css data
df = df[df['ccs_description'].isna()]
print(f'PhD dataset size: {len(df)}')

PhD dataset size: 16844


In [3]:
df_yes_question = df.assign(
    question=df['yes_question'],
    label=1,
)[["task", "question", "label", "image_id", "hitem", "subject", "gt"]]

df_no_question = df.assign(
    question=df['no_question'],
    label=0,
)[["task", "question", "label", "image_id", "hitem", "subject", "gt"]]

df_base = pd.concat([df_yes_question, df_no_question], ignore_index=True)
print("Base PhD dataset size:", len(df_base))
df_base = df_base.sample(frac=1, random_state=42).reset_index(drop=True)
df_base = df_base.reset_index().rename(columns={'index': 'id'})
df_base.head()

Base PhD dataset size: 33688


,id,task,question,label,image_id,hitem,subject,gt
0,0,sentiment,Are the people in the image feeling nostalgic?,0,000000039538,nostalgic,people,happy
1,1,object,Is there a helmet in the image?,0,000000116031,helmet,None,cat
2,2,object,Is there a dishwasher in the image?,0,000000470113,dishwasher,None,oven
3,3,object,Is there a giraffe in the image?,1,000000569718,ostrich,None,giraffe
4,4,counting,Are there five toothbrushes in the image?,1,000000388376,1,toothbrushes,5


In [4]:
dataset_base = df_base.to_dict(orient="records")
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(dataset_base, f, indent=2)